In [ ]:
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('/content/full_dataset_missing_vals_one_row_per_cycle_fw_indexing_48days.csv')

<ipython-input-207-91be9512b258>:1: DtypeWarning: Columns (14,15,16,17,62,63,64,65,110,111,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/full_dataset_missing_vals_one_row_per_cycle_fw_indexing_48days.csv')


In [ ]:
df = df.loc[df['cycle_length']<48]

In [ ]:
df.columns

Index(['hub_id', 'cycle_index', 'cycle_length', 'cycle_status', 'max_LH_day',
       'max_LH_day_bw', 'day0_LH', 'day1_LH', 'day2_LH', 'day3_LH',
       ...
       'day38_PDG', 'day39_PDG', 'day40_PDG', 'day41_PDG', 'day42_PDG',
       'day43_PDG', 'day44_PDG', 'day45_PDG', 'day46_PDG', 'day47_PDG'],
      dtype='object', length=150)

In [ ]:
len(df)

23804

In [ ]:
### Minimizing to just lh
lh_columns = [col for col in df.columns if '_LH' in col]
lh_df = df[['hub_id', 'cycle_index','cycle_status','cycle_length'] + lh_columns]
lh_df_day_cols =['day0_LH', 'day1_LH', 'day2_LH', 'day3_LH', 'day4_LH',
       'day5_LH', 'day6_LH', 'day7_LH', 'day8_LH', 'day9_LH', 'day10_LH',
       'day11_LH', 'day12_LH', 'day13_LH', 'day14_LH', 'day15_LH', 'day16_LH',
       'day17_LH', 'day18_LH', 'day19_LH', 'day20_LH', 'day21_LH', 'day22_LH',
       'day23_LH', 'day24_LH', 'day25_LH', 'day26_LH', 'day27_LH', 'day28_LH',
       'day29_LH', 'day30_LH', 'day31_LH', 'day32_LH', 'day33_LH', 'day34_LH',
       'day35_LH', 'day36_LH', 'day37_LH', 'day38_LH', 'day39_LH', 'day40_LH',
       'day41_LH', 'day42_LH', 'day43_LH', 'day44_LH', 'day45_LH', 'day46_LH',
       'day47_LH']

In [ ]:
"""This is the beginning cycle analysis, forward indexing."""
def fw_idx(hormone_df, day_cols):
    # Total missing days
    hormone_df['total_missing'] = hormone_df[day_cols].apply(lambda x: (x == False).sum(), axis=1)

    # Ranges for beginning analysis
    ranges = {
        '0-3': day_cols[:3],
        '4-6': day_cols[3:6],
        '7-10': day_cols[6:10],
    }

    # Calculate missing days for each range
    for range_name, range_cols in ranges.items():
        hormone_df[f'{range_name} missing'] = hormone_df[range_cols].apply(lambda x: (x == False).sum(), axis=1)

    return hormone_df

df = fw_idx(lh_df, lh_df_day_cols)

<ipython-input-212-4b43ddaa62c1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hormone_df['total_missing'] = hormone_df[day_cols].apply(lambda x: (x == False).sum(), axis=1)
<ipython-input-212-4b43ddaa62c1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hormone_df[f'{range_name} missing'] = hormone_df[range_cols].apply(lambda x: (x == False).sum(), axis=1)
<ipython-input-212-4b43ddaa62c1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [ ]:
def lh_spike_indexing_b(hormone_df, day_cols, start=-6, end=6):
    def calculate_missing(row):
        max_LH_day = int(row['max_LH_day'])
        start_day = max(0, max_LH_day + start)
        end_day = min(len(day_cols), max_LH_day + end + 1)
        vals = row[day_cols[start_day:end_day]]
        return (vals == False).sum()

    hormone_df['missing_around_spike_6_days'] = hormone_df.apply(calculate_missing, axis=1)
    return hormone_df

df = lh_spike_indexing_b(lh_df, lh_df_day_cols)
def lh_spike_indexing_c(hormone_df, day_cols, start=-3, end=3):
    def calculate_missing(row):
        max_LH_day = int(row['max_LH_day'])
        start_day = max(0, max_LH_day + start)
        end_day = min(len(day_cols), max_LH_day + end + 1)
        vals = row[day_cols[start_day:end_day]]
        return (vals == False).sum()

    hormone_df['missing_around_spike_3_days'] = hormone_df.apply(calculate_missing, axis=1)
    return hormone_df
df = lh_spike_indexing_c(df, lh_df_day_cols)

In [ ]:
def new_bw_idx(row, bw_start, bw_end):
  cycle_length = row['cycle_length']

  col_names = [f'day{i}_LH' for i in range(cycle_length + bw_start + 1, cycle_length + bw_end+1) if i>=0]
  days = row[col_names]
  return (days==False).sum()

In [ ]:
new_df = df.apply(lambda x: new_bw_idx(x,bw_start=-3,bw_end=0),axis=1)
new_df

,0
0,3
1,2
2,3
3,3
4,3
...,...
24859,3
24860,3
24861,1
24862,1


In [ ]:
df.columns

Index(['hub_id', 'cycle_index', 'cycle_status', 'cycle_length', 'max_LH_day',
       'max_LH_day_bw', 'day0_LH', 'day1_LH', 'day2_LH', 'day3_LH', 'day4_LH',
       'day5_LH', 'day6_LH', 'day7_LH', 'day8_LH', 'day9_LH', 'day10_LH',
       'day11_LH', 'day12_LH', 'day13_LH', 'day14_LH', 'day15_LH', 'day16_LH',
       'day17_LH', 'day18_LH', 'day19_LH', 'day20_LH', 'day21_LH', 'day22_LH',
       'day23_LH', 'day24_LH', 'day25_LH', 'day26_LH', 'day27_LH', 'day28_LH',
       'day29_LH', 'day30_LH', 'day31_LH', 'day32_LH', 'day33_LH', 'day34_LH',
       'day35_LH', 'day36_LH', 'day37_LH', 'day38_LH', 'day39_LH', 'day40_LH',
       'day41_LH', 'day42_LH', 'day43_LH', 'day44_LH', 'day45_LH', 'day46_LH',
       'day47_LH', 'total_missing', '0-3 missing', '4-6 missing',
       '7-10 missing', 'missing_around_spike_6_days',
       'missing_around_spike_3_days'],
      dtype='object')

In [ ]:
new_df

,0
0,3
1,2
2,3
3,3
4,3
...,...
24859,3
24860,3
24861,1
24862,1


In [ ]:
# Apply the function and assign the result to new columns
bw_start = -3
bw_end = 0
df[f'bw_start_{bw_start}_bw_end_{bw_end}'] = df.apply(lambda x: new_bw_idx(x, bw_start, bw_end), axis=1)

#-6 to -3
bw_start = -6
bw_end = -3
df[f'bw_start_{bw_start}_bw_end_{bw_end}'] = df.apply(lambda x: new_bw_idx(x, bw_start, bw_end), axis=1)

#-9 to -6
bw_start = -9
bw_end = -6
df[f'bw_start_{bw_start}_bw_end_{bw_end}'] = df.apply(lambda x: new_bw_idx(x, bw_start, bw_end), axis=1)

#-12 to -9
bw_start = -12
bw_end = -9
df[f'bw_start_{bw_start}_bw_end_{bw_end}'] = df.apply(lambda x: new_bw_idx(x, bw_start, bw_end), axis=1)

In [ ]:
df.columns

Index(['hub_id', 'cycle_index', 'cycle_status', 'cycle_length', 'max_LH_day',
       'max_LH_day_bw', 'day0_LH', 'day1_LH', 'day2_LH', 'day3_LH', 'day4_LH',
       'day5_LH', 'day6_LH', 'day7_LH', 'day8_LH', 'day9_LH', 'day10_LH',
       'day11_LH', 'day12_LH', 'day13_LH', 'day14_LH', 'day15_LH', 'day16_LH',
       'day17_LH', 'day18_LH', 'day19_LH', 'day20_LH', 'day21_LH', 'day22_LH',
       'day23_LH', 'day24_LH', 'day25_LH', 'day26_LH', 'day27_LH', 'day28_LH',
       'day29_LH', 'day30_LH', 'day31_LH', 'day32_LH', 'day33_LH', 'day34_LH',
       'day35_LH', 'day36_LH', 'day37_LH', 'day38_LH', 'day39_LH', 'day40_LH',
       'day41_LH', 'day42_LH', 'day43_LH', 'day44_LH', 'day45_LH', 'day46_LH',
       'day47_LH', 'total_missing', '0-3 missing', '4-6 missing',
       '7-10 missing', 'missing_around_spike_6_days',
       'missing_around_spike_3_days', 'bw_start_-3_bw_end_0',
       'bw_start_-6_bw_end_-3', 'bw_start_-9_bw_end_-6',
       'bw_start_-12_bw_end_-9'],
      dtype='object')

In [ ]:
missingness_df = df[['hub_id', 'cycle_index', 'cycle_status', 'cycle_length', 'max_LH_day','0-3 missing', '4-6 missing',
       '7-10 missing', 'missing_around_spike_6_days',
       'missing_around_spike_3_days', 'bw_start_-3_bw_end_0',
       'bw_start_-6_bw_end_-3', 'bw_start_-9_bw_end_-6',
       'bw_start_-12_bw_end_-9']]

In [ ]:
missingness_df[['0-3 missing', '4-6 missing',
       '7-10 missing', 'missing_around_spike_6_days',
       'missing_around_spike_3_days', 'bw_start_-3_bw_end_0',
       'bw_start_-6_bw_end_-3', 'bw_start_-9_bw_end_-6',
       'bw_start_-12_bw_end_-9']].describe()

,0-3 missing,4-6 missing,7-10 missing,missing_around_spike_6_days,missing_around_spike_3_days,bw_start_-3_bw_end_0,bw_start_-6_bw_end_-3,bw_start_-9_bw_end_-6,bw_start_-12_bw_end_-9
count,23804.000000,23804.000000,23804.000000,23804.000000,23804.000000,23804.000000,23804.000000,23804.000000,23804.000000
mean,2.612334,2.426525,2.070240,4.787473,1.810788,2.533902,2.171358,1.989792,1.642707
std,0.830929,0.933726,1.489486,3.277704,1.864748,0.770132,1.178802,1.243502,1.287158
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,3.000000,2.000000,1.000000,2.000000,0.000000,2.000000,1.000000,1.000000,0.000000
50%,3.000000,3.000000,2.000000,4.000000,1.000000,3.000000,3.000000,3.000000,2.000000
75%,3.000000,3.000000,4.000000,7.000000,3.000000,3.000000,3.000000,3.000000,3.000000
max,3.000000,3.000000,4.000000,12.000000,6.000000,3.000000,3.000000,3.000000,3.000000


In [ ]:
def classify_rows(df, column_name):
    df[f'classification_{column_name}'] = df[column_name] < 3 # < 2 for 3 days; spike 6 days < 5; spike 3 days < 3
    return df

In [ ]:
def filter_and_print_len(df, column_name):


    filtered_df = df[df[column_name]]

    print(len(filtered_df))

    return filtered_df

In [ ]:
# prompt: download  missingness_df

from google.colab import files
missingness_df.to_csv('missingness_df.csv')
files.download('missingness_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#filter_and_print_len(missingness_df, 'classification_missing_around_spike_6_days')

In [ ]:
### Spike 3 Days has 16142 rows
### Spike 6 Days 6418 rows
### 0-3 days 5280 rows
### 4-6 days 8066 rows
### 7-10 days 14029 rows
### -3 to 0 6998 rows
### -6 to -3 9132 rows
### -9 to -6 10868 rows
### -12 to -9 14227 rows

In [ ]:
# split on one of those features and work to show that the performance is different for people who have more vs less missingness
